In [1]:
import pandas as pd
import numpy as np
import time
# import seaborn as sns
# import matplotlib.pyplot as plt
# import pywt
# import nolds
# import scipy
# import meegkit
import tensorflow as tf
from tensorflow.keras.models import Sequential # do these need to be tf.keras? review why
from tensorflow.keras.layers import Dense, Conv2D, GlobalMaxPooling2D, LSTM, MaxPooling2D, Flatten 
# from sklearn.cross_decomposition import CCA 
from sklearn.model_selection import train_test_split

In [2]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi       2.8Gi       7.0Gi       481Mi       5.7Gi        11Gi
Swap:          976Mi       537Mi       439Mi


In [3]:
# This df will be a row-concatenated version of the same channels
# baseline = pd.read_csv('../../baseline.csv', delimiter=',')
# baseline.drop('Unnamed: 0', inplace=True, axis=1)

In [4]:
occ0 = pd.read_csv('../data/occ0Exp.csv', delimiter=',')
occ1 = pd.read_csv('../data/occ1Exp.csv', delimiter=',')
fefF7 = pd.read_csv('../data/fefF7Exp.csv', delimiter=',')
fefF8 = pd.read_csv('../data/fefF8Exp.csv', delimiter=',')

In [5]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi       5.3Gi       4.5Gi       441Mi       5.7Gi       9.5Gi
Swap:          976Mi       537Mi       439Mi


In [6]:
dfs = [occ0, occ1, fefF7, fefF8]
for i in dfs:
    i.drop('Unnamed: 0', inplace=True, axis=1)
    i['data'].astype(float, copy=False)
    print(i.columns)
#     i = i.groupby('event').mean()
#     print(i.columns)

Index(['event', 'channel', 'code', 'data'], dtype='object')
Index(['event', 'channel', 'code', 'data'], dtype='object')
Index(['event', 'channel', 'code', 'data'], dtype='object')
Index(['event', 'channel', 'code', 'data'], dtype='object')


In [7]:
# for i in full['fefF7Code']:
#     for j in full['fefF8Code']:
#         print(i, j)
        
# it seems the event columns for each channel don't match up, cool..

In [8]:
# non loop version of above's last line

occ0 = occ1.groupby('event').mean()
occ1 = occ1.groupby('event').mean()
fefF7 = fefF7.groupby('event').mean()
fefF8 = fefF8.groupby('event').mean()

In [9]:
for i in range(0, len(dfs)):
    print(len(dfs[i]))

16877097
16877097
16877097
16877097


In [10]:
for i in dfs:
    print(i.isna().sum())

event      0
channel    0
code       0
data       0
dtype: int64
event      0
channel    0
code       0
data       0
dtype: int64
event      0
channel    0
code       0
data       0
dtype: int64
event      0
channel    0
code       0
data       0
dtype: int64


In [11]:
!free -h 

               total        used        free      shared  buff/cache   available
Mem:            15Gi       4.8Gi       5.0Gi       444Mi       5.7Gi         9Gi
Swap:          976Mi       537Mi       439Mi


In [12]:
# putting the var names in a loop that grabs whole dfs is harder than it seems

# for i in dfs:
#     i.rename(columns={'data': f'{i}Data'}, inplace=True)
#     i.rename(columns={'code': f'{i}Code'}, inplace=True)
#     i.rename(columns={'event': f'{i}Event'}, inplace=True)
#     print(i.columns)

occ0.rename(columns={'data': 'occ0Data'}, inplace=True)
occ1.rename(columns={'data': 'occ1Data'}, inplace=True)
fefF7.rename(columns={'data': 'fefF7Data'}, inplace=True)
fefF8.rename(columns={'data': 'fefF8Data'}, inplace=True)

# occ0.rename(columns={'code': 'occ0Code'}, inplace=True)
# occ1.rename(columns={'code': 'occ1Code'}, inplace=True)
# fefF7.rename(columns={'code': 'fefF7Code'}, inplace=True)
# fefF8.rename(columns={'code': 'fefF8Code'}, inplace=True)

# occ0.rename(columns={'event': 'occ0Event'}, inplace=True)
# occ1.rename(columns={'event': 'occ1Event'}, inplace=True)
# fefF7.rename(columns={'event': 'fefF7Event'}, inplace=True)
# fefF8.rename(columns={'event': 'fefF8Event'}, inplace=True)


In [13]:
occ0.columns, occ1.columns, fefF7.columns, fefF8.columns
# dfs[0] #this wasn't changed with the above functions, it's only used for calling the original datasets as they were read in
dfsProcessed = [occ0, occ1, fefF7, fefF8]
dfsProcessed

[        code     occ0Data
 event                    
 67635      6  4214.524654
 67636      7  4221.477317
 67637      9  4221.193910
 67638      9  4220.273892
 67639      0  4219.587771
 ...      ...          ...
 132664    -1  4214.014423
 132665    -1  4210.893429
 132666    -1  4252.604166
 132667    -1  4226.858974
 132668    -1  4227.483974
 
 [65034 rows x 2 columns],
         code     occ1Data
 event                    
 67635      6  4214.524654
 67636      7  4221.477317
 67637      9  4221.193910
 67638      9  4220.273892
 67639      0  4219.587771
 ...      ...          ...
 132664    -1  4214.014423
 132665    -1  4210.893429
 132666    -1  4252.604166
 132667    -1  4226.858974
 132668    -1  4227.483974
 
 [65034 rows x 2 columns],
         code    fefF7Data
 event                    
 67635      6  4496.575936
 67636      7  4494.256410
 67637      9  4461.668669
 67638      9  4494.180264
 67639      0  4459.374753
 ...      ...          ...
 132664    -1  4366.2620

In [14]:
full = pd.concat(dfsProcessed, axis=1, join='inner')

In [15]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi       4.8Gi       5.0Gi       459Mi       5.7Gi         9Gi
Swap:          976Mi       537Mi       439Mi


In [16]:
full.columns.duplicated()
# since the concat created duplicate code columns, using this trick to delete them:
#https://stackoverflow.com/questions/14984119/python-pandas-remove-duplicate-columns
full = full.loc[:,~full.columns.duplicated()].copy()

In [17]:
full.head()

,code,occ0Data,occ1Data,fefF7Data,fefF8Data
event,,,,,
67635,6,4214.524654,4214.524654,4496.575936,3988.708087
67636,7,4221.477317,4221.477317,4494.256410,4005.897436
67637,9,4221.193910,4221.193910,4461.668669,3994.072516
67638,9,4220.273892,4220.273892,4494.180264,3991.991065
67639,0,4219.587771,4219.587771,4459.374753,3991.954635


In [18]:
corrMatFull = full.corr()
corrMatFull
# check = pd.read_csv('../data/occ1Exp.csv', delimiter=',')
# print(check.corr()) # code correlates to data in this channel by 0.001562 
# del check 

# check = pd.read_csv('../data/fefF7Exp.csv', delimiter=',')
# print(check.corr()) # code correlates to data in this channel by -0.004390.
# del check 

# check = pd.read_csv('../data/occ0Exp.csv', delimiter=',')
# print(check.corr()) # code correlates to data in this channel by -0.000308 
# del check 

# check = pd.read_csv('../data/fefF8Exp.csv', delimiter=',')
# print(check.corr()) # code correlates to data in this channel by 0.006184
# del check 

,code,occ0Data,occ1Data,fefF7Data,fefF8Data
code,1.000000,0.001626,0.001626,-0.004799,0.006907
occ0Data,0.001626,1.000000,1.000000,0.136258,0.135879
occ1Data,0.001626,1.000000,1.000000,0.136258,0.135879
fefF7Data,-0.004799,0.136258,0.136258,1.000000,0.172791
fefF8Data,0.006907,0.135879,0.135879,0.172791,1.000000


In [19]:
# full.plot(y=['occ0Data', 'occ1Data', 'fefF7Data', 'fefF8Data'], alpha=.7 ); 


In [20]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi       4.8Gi       5.0Gi       440Mi       5.7Gi         9Gi
Swap:          976Mi       537Mi       439Mi


--------------
## Baseline Model: Only O1, O2, F7, F8; No Preprocessing
## Baseline+ : Basline+Preprocessing

--------------

&nbsp;


In [21]:
X = full.drop('code', axis=1)
y = full['code']
print(X.columns, y.iloc[0], X.shape, y.shape)

Index(['occ0Data', 'occ1Data', 'fefF7Data', 'fefF8Data'], dtype='object') 6 (65034, 4) (65034,)


In [22]:
XTrain, XTest, yTrain, yTest = train_test_split(X, y)

In [23]:
!cowsay brains \& NN algorithms are complex!

 _____________________________________
< brains & NN algorithms are complex! >
 -------------------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||


In [24]:
model = Sequential()

In [25]:

XTrain = XTrain.to_numpy().reshape(-1,4,1)
XTest = XTest.to_numpy().reshape(-1,4,1)
yTrain = yTrain.to_numpy().reshape(-1,1) # do y's need to be reshaped?
yTest = yTest.to_numpy().reshape(-1,1)

In [26]:
XTest.shape

(16259, 4, 1)

In [27]:
model.add(Conv2D(filters=4, kernel_size=1, activation='relu', input_shape=(48775, 4, 1)))
model.add(MaxPooling2D(pool_size=1))
model.add(Conv2D(filters=4, kernel_size=1, activation='relu', input_shape=(48775, 4, 1)))
model.add(MaxPooling2D(pool_size=1))
model.add(Conv2D(filters=4, kernel_size=1, activation='relu', input_shape=(48775, 4, 1)))
model.add(MaxPooling2D(pool_size=1))
model.add(Flatten())
model.add(Dense(12, activation='softmax'))
model.add(Dense(1, activation='softmax'))

# model.add(LSTM(7, input_shape=(1, 4), activation='tanh',
#                recurrent_activation='sigmoid'))
# model.add(LSTM(11, input_shape=(1, 4), activation='tanh',
#                recurrent_activation='sigmoid'))
# model.add(Dense(12, activation='softmax'))
# model.add(Dense(1, activation='softmax'))

#getting same error training both of these models, something is wrong with the input
# since both models produce the same error, it shouldn't be an issue of how the layers made

In [28]:
model.compile(optimizer='Adamax', loss='KLDivergence')
model.fit(XTrain, yTrain, batch_size=7)

ValueError: in user code:

    /home/h/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /home/h/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/h/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/h/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/h/anaconda3/envs/learn-env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/h/anacon

So, I'm getting the same shape error here as I got before I reshaped the training and test sets and changed from Conv1D to Conv2D, but not getting the error when I add the layers like I was, so at least I got one cell farther ^^;

--------------
### Preprocessing
--------------

&nbsp;

Wavelet transform is not ideal for neural data because it relies on windows of different sizes (see Mike X Cohen's youtube channel). Hilbert transform might be the better option
provide unique analytic signal from real value data, so you can calculate instantaneous properties of your data" -EstherExplains v=dy4OeAYqSqM 


HT gives amplitude and phase, so "energy-frequency-time distribution"
> 1, Fourier Transform real valued signal
> 2. Set Fourier coefficients of negative frequencies to zero so they cannot cancel out the imaginary part related to the positive frequencies during the inverse FT
> 3. Double the amplitude related to positive frequencies for energy conservation
> 4. Inverse Fourier transform to obtain the analytic (complex signal)


```
Returns xandarray Analytic signal of x, of each 1-D array along axis
```

In [ ]:
# testing wavelet transform and ways to parse the data column
# pywt.cwt([row.split() for row in emotiv['data']], ) # this one returns array of coefs and an array of freqs 
# testCWT = scipy.signal.wavelets.cwt(dataColFlatFloat, wavelet=scipy.signal.ricker, widths=np.arange(1, 31) ) # this one just returns an NxM matrix, may lose info but could be easier to work with
# type(dataColFlat[3]) # data column is all string encoded, of course, it was one giant string of numbers periods and commas
# dataColFlatFloat = [float(i) for i in dataColFlat]
# pd.DataFrame(testCWT).plot() #freezes comp; reduce dimensionality first

Chaos Features

In [ ]:
# nolds.corr_dim() 
# nolds.lyap_e() # just the exponents
# nolds.lyap_r() # oh here it is

 Power Spectrum Decomposition

In [ ]:
# scipy.signal.periodogram()

Wavelet transform is not ideal for neural data because it relies on windows of different sizes (see Mike X Cohen's youtube channel). Hilbert transform might be the better option
provide unique analytic signal from real value data, so you can calculate instantaneous properties of your data" -EstherExplains v=dy4OeAYqSqM 


HT gives amplitude and phase, so "energy-frequency-time distribution"
> 1, Fourier Transform real valued signal
> 2. Set Fourier coefficients of negative frequencies to zero so they cannot cancel out the imaginary part related to the positive frequencies during the inverse FT
> 3. Double the amplitude related to positive frequencies for energy conservation
> 4. Inverse Fourier transform to obtain the analytic (complex signal)

I think the phase info is in the complex domain though. 
```
Returns xandarray Analytic signal of x, of each 1-D array along axis
```
